In [2]:
!pip install allennlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 730.2/730.2 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.3/776.3 MB 698.6 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.1/19.1 MB 62.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 248.2/248.2 kB 27.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 84.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 49.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 67.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.2/299.2 kB 32.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 7

In [35]:
import numpy as np
import pandas as pd
import torch
from allennlp.modules.elmo import Elmo, batch_to_ids
import jieba

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
cd /content/drive/MyDrive/text_classification/word_vectors/elmo

/content/drive/MyDrive/text_classification/word_vectors/elmo


In [36]:
# Read the Excel file
data = pd.read_excel('./data.xlsx')
sentences = data['review'].tolist()

# Load ELMo model
options_file = "https://s3-us-west-2.amazonaws.com/allennlp/models/elmo/2x4096_512_2048cnn_2xhighway/elmo_2x4096_512_2048cnn_2xhighway_options.json"
weight_file = "https://s3-us-west-2.amazonaws.com/allennlp/models/elmo/2x4096_512_2048cnn_2xhighway/elmo_2x4096_512_2048cnn_2xhighway_weights.hdf5"
elmo_model = Elmo(options_file, weight_file, num_output_representations=1)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

word_vectors = []

# Define a function to tokenize using jieba
def tokenize(text):
    return " ".join(jieba.cut(text))

for sentence in sentences:
    tokens = [tokenize(sentence) for sentence in sentences]
    character_ids = batch_to_ids([tokens]).to(device)

    with torch.no_grad():
        embeddings = elmo_model(character_ids)['elmo_representations'][0]

    word_vector = embeddings.squeeze(0).cpu().numpy()
    word_vectors.append(word_vector)

# Save word vectors to a text file
np.savetxt('word_vectors.txt', np.concatenate(word_vectors))

Building prefix dict from the default dictionary ...
DEBUG:jieba:Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
DEBUG:jieba:Dumping model to file cache /tmp/jieba.cache
Loading model cost 2.168 seconds.
DEBUG:jieba:Loading model cost 2.168 seconds.
Prefix dict has been built successfully.
DEBUG:jieba:Prefix dict has been built successfully.


In [43]:
# get word vector for a specific word
word = "宗教"

# Convert to character IDs
character_id = batch_to_ids([word]).to(device)

with torch.no_grad():
    embeddings = elmo_model(character_id)['elmo_representations'][0]

word_vector = embeddings.squeeze(0).cpu().numpy()

In [44]:
word_vector

array([[-0.4296341 ,  0.        , -0.7186391 , ...,  0.18213691,
        -0.        , -0.        ],
       [-0.1939435 , -0.4741665 , -1.5203922 , ..., -0.5861114 ,
         1.0246979 , -0.30335188]], dtype=float32)

In [45]:
# get dimension
word_vector.shape

(2, 1024)